### VERİYİ TANIMA VE VERİYİ HAZIRLAMA

Veri setimizi modellemeye sokmadan önce, verimizi iyi bir şekilde tanımalı, analiz etmeli ve ona göre bir ön hazırlık gerçekleştirmeliyiz. Verimizi hazırlamak için öncelikle gerekli kütüpanelerimizi ekliyoruz. 

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Gerekli kütüphaneleri ekledikten sonra veri setimizi pandas kütüphanesi yardımıyla okuyoruz ve df değişkenine atıyoruz. 

Ayrıca df.info() ile de veri setimiz hakkında detaylı bilgi sahibi oluyoruz. Veri tipleri, null olmayan değerler ve özellikler, bellek kullanımı gibi değerler verilmiş. 

In [2]:
df = pd.read_excel('/content/Elif_Veri_Kumesi.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   İlan No               739 non-null    object 
 1   Birim Fiyat           779 non-null    float64
 2   Yeni Fiyat            7 non-null      object 
 3   İl                    739 non-null    object 
 4   İlçe                  739 non-null    object 
 5   Mahalle               739 non-null    object 
 6   İlan Tarihi           739 non-null    object 
 7   Güncelleme Tarihi     7 non-null      object 
 8   Konut Şekli           739 non-null    object 
 9   Oda Sayısı            739 non-null    object 
 10  M2                    739 non-null    object 
 11  Bulunduğu Kat         693 non-null    object 
 12  Bina Yaşı             739 non-null    object 
 13  Isınma                739 non-null    object 
 14  Kat Sayısı            682 non-null    object 
 15  Tapu                  3

In [ ]:
df.describe().T

In [ ]:
df

Verimizi gösteren çıktıdan da anlaşılacağı gibi modelimizi eğitirken, ham veri kümemizde bulunan birçok sütuna (özelliğe) ihtiyacımız olmayacaktır. Çünkü ilan tarihi, ilan no, yeni fiyat, il, ilçe, mahalle gibi özellikler modelimizin fiyat tahmini yapmasında kullanılmayacaktır. Bu nedenle de ham veri setimiz işleme tabi tutulmadan önce gerekli olan ön işlemleri yapmamız gerekiyor.

In [3]:
df.drop("İlan No", axis = 1, inplace = True)
df.drop("Yeni Fiyat", axis = 1, inplace = True)
df.drop("İl", axis = 1, inplace = True)
df.drop("İlçe", axis = 1, inplace = True)
df.drop("Mahalle", axis = 1, inplace = True)
df.drop("İlan Tarihi", axis = 1, inplace = True)
df.drop("Güncelleme Tarihi", axis = 1, inplace = True)
df.drop("Cephe", axis = 1, inplace = True)
df.drop("tahmini_satış_fiyatı", axis = 1, inplace = True)

Şimdi ise geri kalan verilerimizdeki düzeltmeleri yapmamız gerekiyor. Örneğin veri setimizde m2 de alan ve net metre kare olarak yazılmış. Bunu düzgünce ayırıp gereksiz olan m2 yazılarını silmeliyiz. Ayrıca iki tane m2 değeri olduğundan birini silmemiz daha iyi olacaktır.  

In [4]:
df["M2"] = df.M2.str[:-3]
df["M2"] = df.M2.str[-3:]

Bina yaşı kısmında ise 10 yaşında 25 yaşında gibi veriler var bunları daha doğru yorumlayabilmek açısından yaşında kısımlarını siliyoruz. Ayrıca sıfır bina şeklinde yazanları ise 0 şeklinde değiştirmeliyiz. Bunları gerçekleştirmek için aşağıdaki hücreyi yazıyoruz. Bu yaptığım düzenlemeleri aynı şekilde Kat Sayısı, ve Bulunduğu Kat ifadelerine de uyguluyorum. Böylece bu değerleri sayısal olarak kullanabileceğiz.

In [5]:
df.loc[df['Bina Yaşı'] == "Sıfır Bina", "Bina Yaşı"] = "0 Yaşında"
df["Bina Yaşı"] = df["Bina Yaşı"].str[:-8]

Daha sonra ise gözden kaçırmış olabileceğim verileri anlamak ve farklı değerler varsa değiştirmek adına value_counts() fonksiyonu ile hepsini inceliyorum. Bulunan 11 tane özelliğe baktığımızda birçoğunda hatalı veriler de var, bu hatalı verileri elimizden geldiğince temizlemeliyiz ki modelimiz daha doğru bir sonuç versin. 

In [ ]:
df

In [ ]:
df["Konut Şekli"].value_counts()

Konut Şekli çıktısında herhangi değiştirilecek bir değer yok. Bu yüzden bir işlem yapmıyoruz.

In [ ]:
df["Oda Sayısı"].value_counts()

Oda sayısını ileride daha rahat kullanabilmek için oda sayısı cinsinden yazmamız daha iyi olacaktır, çünkü geneldeki evlerin bir salonları var. Ayrıca 30 odalı bir ev olamayacağından bu gibi değerleri değiştirmeliyiz.

In [6]:
df["Oda Sayısı"] = df["Oda Sayısı"].str[:-3]
df.loc[df['Oda Sayısı'] == "15 +", "Oda Sayısı"] = 2
df.loc[df['Oda Sayısı'] == "30 +", "Oda Sayısı"] = 3

In [ ]:
df["M2"].value_counts()

In [ ]:
df["Bulunduğu Kat"].value_counts()

In [ ]:
df["Bina Yaşı"].value_counts()

In [ ]:
df["Isınma"].value_counts()

In [ ]:
df["Kat Sayısı"].value_counts()

In [ ]:
df["Tapu"].value_counts()

In [ ]:
df["Yakıt Tipi"].value_counts()

Üst kısımdaki hücreleri çalıştırdığımızda: Konut Şekli, Oda Sayısı, Kat Sayısı, M2, Tapu ve Yakıt verilerinde bir hata olmadığını görüyoruz. Fakat Bulunduğu kat verilerinde giriş, çatı, villa gibi değerler var bu değerleri düzeltmemiz gerekiyor. Ayrıca Isınma sutünundaki verilerde belirtilmemiş olan değeri de en çok kullanılan değerle değiştiriyoruz.

In [9]:
df.loc[df['Bulunduğu Kat'] == "Yüksek Giriş", "Bulunduğu Kat"] = "1000000"
df.loc[df['Bulunduğu Kat'] == "Zemin", "Bulunduğu Kat"] = "000000"
df.loc[df['Bulunduğu Kat'] == "En Üst Kat", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "", "Bulunduğu Kat"] = "3. Kat"
df.loc[df['Bulunduğu Kat'] == "Bahçe", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Giriş", "Bulunduğu Kat"] = "0. Kat"
df.loc[df['Bulunduğu Kat'] == "Teras", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Yüksek Giriş", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Villa", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Çatı", "Bulunduğu Kat"] = df["Kat Sayısı"]
df.loc[df['Bulunduğu Kat'] == "Ar", "Bulunduğu Kat"] = "3. Kat"
df.loc[df['Bulunduğu Kat'] == "Yarı Bodrum", "Bulunduğu Kat"] = df["Kat Sayısı"]
df["Bulunduğu Kat"] = df["Bulunduğu Kat"].str[:-5]
df["Kat Sayısı"] = df["Kat Sayısı"].str[:-5]

df.loc[df['Isınma'] == "Belirtilmemiş", "Isınma"] = "Kombi"

In [10]:
df

,Birim Fiyat,Konut Şekli,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Isınma,Kat Sayısı,Tapu,Yakıt Tipi
0,3166.666700,Daire,2,75,10,10,Kombi,4,NaN,Doğalgaz
1,3000.000000,Daire,3,110,8,25,Merkezi,8,NaN,Kömür-Odun
2,3000.000000,Daire,2,90,2,25,Kombi,6,NaN,Doğalgaz
3,4172.727300,Daire,3,90,10,10,Kombi,5,NaN,Doğalgaz
4,4450.000000,Daire,2,90,1,0,Merkezi,7,Kat İrtifakı,Doğalgaz
...,...,...,...,...,...,...,...,...,...,...
774,11.538500,Daire,1,60,3,2,Kombi,4,NaN,Doğalgaz
775,10.833300,Daire,1,55,3,4,Merkezi,4,NaN,Doğalgaz
776,13.238706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,16.666700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Yukarıdaki hücre çalıştırıldığında görüldüğü üzere Tapu alanında bir çok NaN değer var bu değerleri en çok kullanılmış tapu verisi ile yani modu ile değiştirmemiz gerekiyor. Bu Nan değerler tüm sutünlarda olabileceğinden bazı sütunlara bu fonksiyonu uyguluyoruz.

In [ ]:
df["Konut Şekli"] = df["Konut Şekli"].fillna(df["Konut Şekli"].mode()[0])
df["Oda Sayısı"] = df["Oda Sayısı"].fillna(df["Oda Sayısı"].mode()[0])
df["Bina Yaşı"] = df["Bina Yaşı"].fillna(df["Bina Yaşı"].mode()[0])
df["Bulunduğu Kat"] = df["Bulunduğu Kat"].fillna(df["Bulunduğu Kat"].mode()[0])
df["Isınma"] = df["Isınma"].fillna(df["Isınma"].mode()[0])
df["Bulunduğu Kat"] = df["Bulunduğu Kat"].fillna(df["Bulunduğu Kat"].mode()[0])
df["Tapu"] = df["Tapu"].fillna(df["Tapu"].mode()[0])
df["Yakıt Tipi"] = df["Yakıt Tipi"].fillna(df["Yakıt Tipi"].mode()[0])

Ve son olarak NaN değerlerimizin tümünü dropna fonksiyonu sayesinde sildik.

In [13]:
df = df.dropna()

### KATEGORİK DEĞERLERİN ETİKETLENMESİ

Burada bulunan kategorik değişkenlerimizi etiketlemek için önce sklearn’deki preprocessing modülünü kullanıyoruz. Sonrasında ise yeni bir LabelEncoder() nesnesi üretiyoruz. Kategorik verilerimiz olan Konut Şekli, Isınma, Tapu ve Yakıt tipini fit_transform ile etiketliyoruz.  

In [14]:
df

,Birim Fiyat,Konut Şekli,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Isınma,Kat Sayısı,Tapu,Yakıt Tipi
4,4450.0000,Daire,2,90,1,0,Merkezi,7,Kat İrtifakı,Doğalgaz
5,4032.2581,Daire,3,130,7,11,Kombi,7,Kat Mülkiyeti,Doğalgaz
6,2666.6667,Daire,3,130,6,18,Kombi,6,Kat Mülkiyeti,Doğalgaz
7,2175.0000,Daire,4,180,4,9,Kombi,4,Kat İrtifakı,Doğalgaz
8,2481.4815,Daire,3,130,3,30,Merkezi (Pay Ölçer),6,Kat Mülkiyeti,Kömür-Odun
...,...,...,...,...,...,...,...,...,...,...
742,4428.5714,Daire,3,125,2,0,Kombi,5,Kat İrtifakı,Doğalgaz
743,3448.2759,Daire,1,48,10,0,Kombi,4,Kat İrtifakı,Doğalgaz
744,4107.1429,Daire,2,100,3,5,Kombi,4,Kat Mülkiyeti,Doğalgaz
748,3000.0000,Daire,1,35,3,5,Kombi,3,Kat Mülkiyeti,Doğalgaz


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

df['Isınma Tipi'] = labelencoder.fit_transform(df['Isınma'])
df.drop("Isınma", axis = 1, inplace = True)

df['Konut Tipi'] = labelencoder.fit_transform(df['Konut Şekli'])
df.drop("Konut Şekli", axis = 1, inplace = True)

df['Yakıt Şekli'] = labelencoder.fit_transform(df['Yakıt Tipi'])
df.drop("Yakıt Tipi", axis = 1, inplace = True)

df['Tapu Tipi'] = labelencoder.fit_transform(df['Tapu'])
df.drop("Tapu", axis = 1, inplace = True)

In [18]:
df

,Birim Fiyat,Oda Sayısı,M2,Bulunduğu Kat,Bina Yaşı,Kat Sayısı,Isınma Tipi,Konut Tipi,Yakıt Şekli,Tapu Tipi
4,4450.0000,2,90,1,0,7,2,1,0,2
5,4032.2581,3,130,7,11,7,1,1,0,1
6,2666.6667,3,130,6,18,6,1,1,0,1
7,2175.0000,4,180,4,9,4,1,1,0,2
8,2481.4815,3,130,3,30,6,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...
742,4428.5714,3,125,2,0,5,1,1,0,2
743,3448.2759,1,48,10,0,4,1,1,0,2
744,4107.1429,2,100,3,5,4,1,1,0,1
748,3000.0000,1,35,3,5,3,1,1,0,1


Bu işlem sonrası artık tüm verilerimiz numerik hale geldi. Hazırladığımız bu veri setini kaydedip bir sonraki aşamamız olan modellemeye geçebiliriz.

In [17]:
df.to_excel("Veri_Kumesi_Edited.xlsx")